In [23]:
import numpy as np
import cupy as cu
import cupyx.scipy.ndimage
import json

In [5]:
import czifile
_testFile = czifile.CziFile('sample_data/RBC_tiny.czi')


In [53]:

_testFile.metadata().find('Scale')

36533

In [56]:
#_testFile.metadata()

In [57]:
image  = _testFile.asarray()[0, 0, :,:,:,0]

In [58]:
import tifffile
image = tifffile.imread('sample_data/RBC_lattice.tif')

In [59]:
print(image.shape)
depth, height, original_width = image.shape


(150, 118, 209)


In [90]:
import cupy as cp
import cupyx.scipy.ndimage
import numpy as np

# Constants
z_pixel_size = 0.3  # Microns
y_pixel_size = 0.1449  # Microns
angle_in_radians = np.radians(30)  # Convert angle to radians
depth, height, original_width = image.shape
print(image.shape)


def calculate_new_dimensions_and_offset_x(depth, original_width, angle_in_radians, z_pixel_size, x_pixel_size):
    # Calculate the shear factor for the last slice in the stack, which will be the maximum
    max_shear_factor = calculate_shear_factor(depth - 1, angle_in_radians, z_pixel_size, x_pixel_size)
    
    # Calculate the new width which includes the original width plus the maximum shear displacement
    new_width = int(original_width + np.abs(max_shear_factor))
    
    # The offset for the affine transform to center the sheared image
    offset_x = max_shear_factor / 2
    
    return new_width, offset_x

# The calculate_shear_factor function remains the same, as it is agnostic to the direction of shear

# Calculate the new dimensions and offset for the entire stack in the X direction

new_width, offset_x = calculate_new_dimensions_and_offset_x(depth, original_width, angle_in_radians, z_pixel_size, y_pixel_size)
print(new_width)
# Initialize the de-skewed volume with the new width and same height as the original
deskewed_volume = cp.zeros((depth, new_width, height), dtype=cp.float32)  # Ensure dtype matches your image
print(deskewed_volume.shape)
for z in range(depth):
    # Calculate slice-specific shear factor
    slice_shear = calculate_shear_factor(z, angle_in_radians, z_pixel_size, y_pixel_size)
    
    # Define the transformation matrix for this slice with the shear factor in the X direction
    transformation_matrix = cp.array([
        [1, slice_shear, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
    
    # Apply the affine transformation
    # Offset is only used in the x-direction, and it's negative because we are moving the image back
    deskewed_volume[z, :, :] = cupyx.scipy.ndimage.affine_transform(
        cp.asarray(image[z, :, :]),  # Ensure image is a CuPy array
        transformation_matrix[:2, :2],  # Use the 2x2 transformation matrix
        offset=[-offset_x, 0],  # Offset to center the image
        output_shape=(new_width, original_height),
        order=1,
        mode='constant',
        cval=0.0
    )

(150, 118, 209)
387
(150, 387, 118)


In [101]:

# Re-defining the constants and function after the reset
z_pixel_size = 0.3  # Microns
y_pixel_size = 0.1449  # Microns
angle_in_radians = np.radians(30)  # Convert angle to radians

def calculate_shear_factor(z, angle_in_radians, z_pixel_size, y_pixel_size):
    # Calculate the physical offset in the Y direction for the current Z index
    physical_offset_y = z * z_pixel_size * np.tan(angle_in_radians)
    # Convert the physical offset to pixels in the Y dimension
    shear_factor = physical_offset_y / y_pixel_size
    return shear_factor

# Test the function for the first few Z slices
shear_factors = [calculate_shear_factor(z, angle_in_radians, z_pixel_size, y_pixel_size) for z in range(5)]
shear_factors

def calculate_new_dimensions_and_offset(depth, original_width, angle_in_radians, z_pixel_size, y_pixel_size):
    # Calculate the shear factor for the last slice in the stack, which will be the maximum
    max_shear_factor = calculate_shear_factor(depth - 1, angle_in_radians, z_pixel_size, y_pixel_size)
    
    # Calculate the new width which includes the original width plus the maximum shear displacement
    new_width = int(original_width + np.abs(max_shear_factor))
    
    # The offset for the affine transform to center the sheared image
    offset_y = max_shear_factor / 2
    
    return new_width, offset_y

# Let's calculate the new dimensions and offset for the entire stack
original_width = 209  # Example width, replace with your actual image width
depth = 150  # Example depth, replace with your actual image depth
new_width, offset_y = calculate_new_dimensions_and_offset(depth, original_width, angle_in_radians, z_pixel_size, y_pixel_size)

print(f"New width: {new_width}, Offset Y: {offset_y}")


New width: 387, Offset Y: 89.05299183152611


In [102]:
import cupy as cp
import cupyx.scipy.ndimage

# Assume 'image' is your input 3D array with shape (depth, height, width)
# Initialize the de-skewed volume with the new width and same height as the original
deskewed_volume = cp.zeros((depth, original_height, new_width), dtype=cp.float32)  # Ensure dtype matches your image

for z in range(depth):
    # Calculate slice-specific shear factor
    slice_shear = calculate_shear_factor(z, angle_in_radians, z_pixel_size, y_pixel_size)
    
    # Define the transformation matrix for this slice with the shear factor
    transformation_matrix = cp.array([
        [1, 0, 0],
        [slice_shear, 1, 0],
        [0, 0, 1]
    ])
    
    # Apply the affine transformation
    # Offset is only used in the y-direction, and it's negative because we are moving the image back
    deskewed_volume[z, :, :] = cupyx.scipy.ndimage.affine_transform(
        cp.asarray(image[z, :, :]),  # Ensure image is a CuPy array
        transformation_matrix[:2, :2],  # Use the 2x2 transformation matrix
        offset=[0, -offset_y],  # Offset to center the image
        output_shape=(original_height, new_width),
        order=1,  # Linear interpolation
        mode='constant',
        cval=0.0  # Background value
    )

In [95]:

def calculate_new_dimensions(original_width, depth, shear_factor, z_pixel_size):
    # Calculate the new width by considering the maximum shear across the Z depth
    # The depth in pixels multiplied by the Z pixel size gives the total physical depth
    # Multiplying by the tangent of the excitation angle gives the total shear in microns
    # Dividing by the Y pixel size converts that shear back into Y pixels
    max_skew_in_pixels = (depth * z_pixel_size * cu.tan(angle_in_radians)) / y_pixel_size
    new_width = int(original_width + cu.abs(max_skew_in_pixels))
    return new_width


# Physical dimensions of pixels
y_pixel_size = 0.1449  # in microns
z_pixel_size = 0.3    # in microns

# Calculate the shear factor based on the physical dimensions
# We want to know how many Z pixels correspond to one Y pixel
shear_factor_per_pixel = y_pixel_size / z_pixel_size

# Convert the excitation angle to radians and calculate the shear factor
angle_in_radians = cu.radians(30)
shear_factor = cu.tan(angle_in_radians) * shear_factor_per_pixel


# Define the transformation matrix for de-skewing
# This matrix represents a shear parallel to the Y-axis
transformation_matrix = cu.array([[1, shear_factor.get(), 0],
                                  [0, 1, 0]])


# Define the transformation matrix for de-skewing each slice
transformation_matrix = cu.array([[1, shear_factor.get(), 0],
                                  [0, 1, 0]])

depth, original_height, original_width = image.shape
max_physical_skew = depth * z_pixel_size * cu.tan(angle_in_radians)
max_skew_in_pixels = max_physical_skew / y_pixel_size
new_width = int(original_width + cu.abs(max_skew_in_pixels))

# Create a new volume to hold the de-skewed images
deskeWed_volume = cu.zeros((depth, original_height, new_width), dtype=image.dtype)
image = cu.asarray(image)
for z in range(depth):
    # Calculate the slice-specific shear factor based on the Z index
    # Currently I don't think this is calculated correctly so I am using the skear_factor
    slice_shear_factor = shear_factor * z* z_pixel_size

    # Apply the 2D affine transformation to each slice along the Z-axis
    transformation_matrix = cu.array([[1, shear_factor.get(), 0],
                                  [0, 1, 0]])


    # Apply the affine transformation using the upper 2x2 matrix for each slice
    deskeWed_volume[z, :, :] = cupyx.scipy.ndimage.affine_transform(
        image[z, :, :],
        transformation_matrix[:2, :2],
        offset=0,
        output_shape=(original_height, new_width),
        order=1,
        mode='constant',
        cval=0.0)

In [96]:
import numpy as np
import cupy as cp
import cupyx.scipy.ndimage

def calculate_new_dimensions(depth, original_width, angle_in_radians, z_pixel_size, y_pixel_size):
    # Calculate the maximum skew in pixels
    max_skew_in_microns = depth * z_pixel_size * np.tan(angle_in_radians)
    max_skew_in_pixels = max_skew_in_microns / y_pixel_size
    new_width = int(original_width + np.abs(max_skew_in_pixels))
    return new_width

# Constants
y_pixel_size = 0.1449  # microns
z_pixel_size = 0.3     # microns
angle_in_radians = np.radians(30)  # Convert angle to radians

# Image dimensions (example)
depth, original_height, original_width = (150, 118, 209)  # Update with your image.shape

# Calculate new dimensions
new_width = calculate_new_dimensions(depth, original_width, angle_in_radians, z_pixel_size, y_pixel_size)

# Initialize the de-skewed volume
deskeWed_volume = cp.zeros((depth, original_height, new_width), dtype=cp.float32)  # Ensure dtype matches your image

for z in range(depth):
    # Calculate slice-specific shear factor
    slice_shear = (z * z_pixel_size * np.tan(angle_in_radians)) / y_pixel_size
    
    # Define the transformation matrix for this slice
    transformation_matrix = cp.array([
        [1, slice_shear, 0],
        [0, 1, 0],
        [0, 0, 1]
    ])
    
    # Apply the affine transformation
    # Make sure to pass only the necessary 2x2 portion of the transformation matrix
    # and correctly format the offset if used
    deskeWed_volume[z, :, :] = cupyx.scipy.ndimage.affine_transform(
        cp.asarray(image[z, :, :]),  # Ensure image is a CuPy array
        transformation_matrix,  # Correctly formatted 2x2 transformation matrix
        offset=0,  # Ensure offset is correctly formatted
        output_shape=(original_height, new_width),
        order=1,
        mode='constant',
        cval=0.0
    )



In [108]:
import cupy as cp
import cupyx.scipy.ndimage
import numpy as np

# Constants from your code
voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3
deskewing_angle_in_degrees = 30
angle_in_radians = np.radians(deskewing_angle_in_degrees)

# Original image shape
depth, original_height, original_width = (150, 118, 209)

# Calculate the new dimensions and offset for the entire stack in the Y direction
def calculate_new_dimensions_and_offset_y(depth, original_height, angle_in_radians, voxel_size_z_in_microns, voxel_size_y_in_microns):
    max_shear_factor = calculate_shear_factor(depth - 1, angle_in_radians, voxel_size_z_in_microns, voxel_size_y_in_microns)
    new_height = int(original_height + np.abs(max_shear_factor))
    offset_y = max_shear_factor / 2
    return new_height, offset_y

# Calculate the shear factor as before
def calculate_shear_factor(z, angle_in_radians, voxel_size_z, voxel_size_y):
    physical_offset_y = z * voxel_size_z * np.tan(angle_in_radians)
    shear_factor = physical_offset_y / voxel_size_y
    return shear_factor

new_height, offset_y = calculate_new_dimensions_and_offset_y(depth, original_height, angle_in_radians, voxel_size_z_in_microns, voxel_size_y_in_microns)

# Initialize the de-skewed volume with the new height and the same width as the original
deskewed_volume = cp.zeros((depth, new_height, original_width), dtype=cp.float32)

for z in range(depth):
    # Calculate slice-specific shear factor for the Y coordinates
    slice_shear = calculate_shear_factor(z, angle_in_radians, voxel_size_z_in_microns, voxel_size_y_in_microns)
    
    # Define the transformation matrix for this slice with the shear factor in the Y direction
    transformation_matrix = cp.array([
        [1, 0, 0],
        [slice_shear, 1, 0],
        [0, 0, 1]
    ])
    
    # Apply the affine transformation to each 2D slice in the XY plane
    deskewed_volume[z, :, :] = cupyx.scipy.ndimage.affine_transform(
        cp.asarray(image[z, :, :]),  # Ensure image is a CuPy array
        transformation_matrix[:2, :2],  # Use the 2x2 transformation matrix
        offset=[0, -offset_y],  # Offset to center the image in the new height
        output_shape=(new_height, original_width),
        order=1,
        mode='constant',
        cval=0.0
    )

In [109]:
print(deskewed_volume.shape)
print(image.shape)


(150, 295, 209)
(150, 118, 209)


In [98]:
def rotate_volume(deskeWed_volume, angle_of_rotation):
    rotated_volume = cu.zeros_like(deskeWed_volume)
    
    # Rotate each slice of the volume
    for z in range(deskeWed_volume.shape[0]):
        # Rotate the 2D slice around the Z-axis (the axis perpendicular to the XY plane)
        rotated_volume[z, :, :] = cupyx.scipy.ndimage.rotate(
            deskeWed_volume[z, :, :],
            angle_of_rotation,
            axes=(1, 0),  # This rotates the slice in the XY plane
            reshape=False,
            order=1,
            mode='nearest',
            cval=0.0
        )
    
    return rotated_volume

# Apply the rotation to the de-skewed volume
angle_of_rotation = 90  # or whatever angle is needed to align with the stage coordinates
rotated_volume = rotate_volume(deskeWed_volume, angle_of_rotation)

In [111]:
for z in range(depth):
    # Calculate slice-specific shear factor for the X coordinates
    slice_shear = calculate_shear_factor(z, angle_in_radians, z_pixel_size, x_pixel_size)
    
    # Define the transformation matrix for this slice with the shear factor in the X direction
    transformation_matrix = cp.array([
        [1, 0, 0],
        [0, 1, 0],
        [slice_shear, 0, 1]  # Shear in the X direction relative to Z
    ])
    
    print(slice_shear)

0.0
1.1953421722352497
2.3906843444704995
3.5860265167057497
4.781368688940999
5.97671086117625
7.172053033411499
8.367395205646751
9.562737377881998
10.758079550117248
11.9534217223525
13.148763894587749
14.344106066822999
15.539448239058249
16.734790411293503
17.93013258352875
19.125474755763996
20.320816927999246
21.516159100234496
22.71150127246975
23.906843444705
25.10218561694025
26.297527789175497
27.492869961410747
28.688212133645997
29.883554305881248
31.078896478116498
32.27423865035175
33.469580822587005
34.66492299482225
35.8602651670575
37.05560733929274
38.25094951152799
39.44629168376325
40.64163385599849
41.83697602823375
43.03231820046899
44.22766037270425
45.4230025449395
46.61834471717474
47.81368688941
49.00902906164524
50.2043712338805
51.399713406115744
52.595055578350994
53.790397750586244
54.985739922821494
56.18108209505675
57.376424267291995
58.57176643952724
59.767108611762495
60.96245078399774
62.157792956232996
63.35313512846824
64.5484773007035
65.74381947

In [105]:
import cupy as cp
import cupyx.scipy.ndimage
import numpy as np

# Constants
z_pixel_size = 0.3  # Microns
x_pixel_size = 0.1449  # Assuming X and Y have the same pixel size
angle_in_radians = np.radians(30)  # Convert angle to radians
original_height = 118  # Example height, replace with your actual image height
original_width = 209  # Example width, replace with your actual image width
depth = 150  # Example depth, replace with your actual image depth

def calculate_shear_factor(z, angle_in_radians, z_pixel_size, x_pixel_size):
    # Calculate the physical offset in the X direction for the current Z index
    physical_offset_x = z * z_pixel_size * np.tan(angle_in_radians)
    # Convert the physical offset to pixels in the X dimension
    shear_factor = physical_offset_x / x_pixel_size
    return shear_factor

# Initialize the de-skewed volume with the same dimensions as the original
deskewed_volume = cp.zeros((depth, original_height, original_width), dtype=cp.float32)  # Ensure dtype matches your image

for z in range(depth):
    # Calculate slice-specific shear factor for the X coordinates
    slice_shear = calculate_shear_factor(z, angle_in_radians, z_pixel_size, x_pixel_size)
    
    # Define the transformation matrix for this slice with the shear factor in the X direction
    transformation_matrix = cp.array([
        [1, 0, 0],
        [0, 1, 0],
        [slice_shear, 0, 1]  # Shear in the X direction relative to Z
    ])
    
    # Apply the affine transformation to each 2D slice in the XZ plane
    deskewed_volume[z, :, :] = cupyx.scipy.ndimage.affine_transform(
        cp.asarray(image[z, :, :]),  # Ensure image is a CuPy array
        transformation_matrix[:2, :2],  # Use the 2x2 transformation matrix
        offset=[0, 0],  # No offset needed since we are not changing the width
        output_shape=(original_height, original_width),
        order=1,
        mode='constant',
        cval=0.0
    )


In [110]:
import napari
views = napari.Viewer()

views.add_image(deskewed_volume.get(), scale=(.3, .15,.15), name='deskewed')
views.add_image(image.get(), scale=(.3, .15,.15))

<Image layer 'Image' at 0x7fc23bc7c1c0>

In [71]:
views.add_image(rotated_volume.get(), scale=(.3, .15,.15))

<Image layer 'Image [1]' at 0x7fc2682cbb50>

In [85]:
image2= tifffile.imread('sample_data/RBC_lattice.tif')
views.add_image(image2, scale=(.3, .15,.15))

<Image layer 'image2' at 0x7f58146899f0>

In [25]:
image_volume = image
angle_in_degrees = 30
y_pixel_size = 0.15
z_pixel_size = 0.3

depth, height, original_width = image_volume.shape

# Calculate the shear factor based on the excitation angle and pixel sizes
angle_in_radians = cu.radians(angle_in_degrees)
shear_factor = cu.tan(angle_in_radians) * (y_pixel_size / z_pixel_size)

# Calculate the new width to accommodate the shearing
max_skew_in_pixels = (depth - 1) * cu.tan(angle_in_radians) * (z_pixel_size / y_pixel_size)
new_width = int(original_width + cu.abs(max_skew_in_pixels))

# Create a new volume to hold the de-skewed images
deskeWed_volume = cu.zeros((depth, height, new_width), dtype=image_volume.dtype)

# Define the 4x4 affine transformation matrix for 3D shearing in y over z
transformation_matrix = cu.array([
    [1, 0, 0, 0],
    [shear_factor.get(), 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Perform the affine transformation on the entire volume
deskeWed_volume = cupyx.scipy.ndimage.affine_transform(
    image_volume,
    transformation_matrix,
    offset=0,
    output_shape=(depth, height, new_width),
    order=1,
    mode='constant',
    cval=0.0
)



In [67]:
import napari
views = napari.Viewer()
views.add_image(image_volume.get(), scale=(.3, .15,.15))
views.add_image(deskeWed_volume.get(), scale=(.3, .15,.15))

NameError: name 'image_volume' is not defined